In [24]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.random.set_seed(100)

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

df = sns.load_dataset('mpg')
print(df.shape)
df.describe(include='all')

(398, 9)


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000,398,398
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,305
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,usa,ford pinto
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,249,6
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050,NaN,NaN
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627,NaN,NaN
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,NaN,NaN
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000,NaN,NaN
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000,NaN,NaN
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000,NaN,NaN


In [25]:
df['horsepower'].fillna(df['horsepower'].median(), inplace=True)
df = df.drop(['name'], axis=1)
df = pd.get_dummies(df, columns=['cylinders'], drop_first=True, prefix='Cylinder')
df = pd.get_dummies(df, columns=['model_year'], drop_first=True, prefix='Year')
df = pd.get_dummies(df, columns=['origin'], drop_first=True, prefix='Origin')
df.head()

C:\Users\jdvis_x5odeao\AppData\Local\Temp\ipykernel_22560\3318664815.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['horsepower'].fillna(df['horsepower'].median(), inplace=True)


,mpg,displacement,horsepower,weight,acceleration,Cylinder_4,Cylinder_5,Cylinder_6,Cylinder_8,Year_71,...,Year_75,Year_76,Year_77,Year_78,Year_79,Year_80,Year_81,Year_82,Origin_japan,Origin_usa
0,18.0,307.0,130.0,3504,12.0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
1,15.0,350.0,165.0,3693,11.5,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
2,18.0,318.0,150.0,3436,11.0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
3,16.0,304.0,150.0,3433,12.0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
4,17.0,302.0,140.0,3449,10.5,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True


In [26]:
target_variable = ['mpg'] 
predictors = list(set(list(df.columns))-set(target_variable))
df[predictors] = df[predictors]/df[predictors].max()
df.describe()

,mpg
count,398.000000
mean,23.514573
std,7.815984
min,9.000000
25%,17.500000
50%,23.000000
75%,29.000000
max,46.600000


In [27]:
X = df[predictors].values
y = df[target_variable].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

print(X_train.shape); print(X_test.shape)

(278, 22)
(120, 22)


In [28]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32, input_shape = (X_train.shape[1],), activation = 'relu'))
model.add(tf.keras.layers.Dense(16, activation= "relu"))
model.add(tf.keras.layers.Dense(8, activation= "relu"))
model.add(tf.keras.layers.Dense(4, activation= "relu"))
model.add(tf.keras.layers.Dense(1))    

c:\Users\jdvis_x5odeao\Semester_4\DEDS_Portfolio\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss = 'mae', metrics = ['mae'], optimizer = optimizer)  


In [30]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')


model.fit(X_train, y_train, epochs=50)

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 22.7110 - mae: 22.7110
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 15.3025 - mae: 15.3025 
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.5714 - mae: 8.5714   
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.5656 - mae: 4.5656 
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.7036 - mae: 3.7036 
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.1675 - mae: 3.1675 
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7806 - mae: 2.7806 
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.8665 - mae: 2.8665 
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.6691 - mae: 2.6691 
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.6221 - mae: 2.6221 
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.8135 - mae: 2.8135 
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7954 - mae: 2.7954 
Epoch 13/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss:

In [31]:
print(model.evaluate(X_train, y_train))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8388 - mae: 1.8388 
[1.7977323532104492, 1.7977323532104492]


In [32]:
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

print(model.evaluate(X_test, y_test))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.2308 - mae: 2.2308
[2.405280113220215, 2.405280113220215]


In [35]:
pred_train = model.predict(X_train)
r2_score(y_train, pred_train)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


0.8966689109802246

In [34]:
pred_test = model.predict(X_test)
r2_score(y_test, pred_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


0.8073281049728394